In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from encode_data import *
from midi_data import *

In [28]:
from tqdm import tqdm

## Encode music21 stream to text representation and back

### Load midi data

In [3]:
path = Path('../data/midi')
hook_path = path/'hooktheory'

In [4]:
import pandas as pd

In [6]:
df = pd.read_csv('hooktheory_metadata.csv'); df.head()

,parts,ht_mode,ht_key,original_path,genres,seconds,section,midi_keyc,time_signature,ht_time_signature,ht_bpm,instruments,bpm,midi_title,song_url,artist,inferred_key,inferred_keyc
0,"['intro', 'chorus']",1.0,C,../data/midi/hooktheory/pianoroll/w/wayne-shar...,[],15.000000,chorus,../data/midi/transposed/hooktheory/pianoroll/w...,4/4,4,128,"['Piano', 'Piano']",128.0,yu-gi-oh3,https://www.hooktheory.com/theorytab/view/wayn...,wayne-sharpe,C major,C major
1,"['intro', 'chorus']",1.0,C,../data/midi/hooktheory/pianoroll/w/wayne-shar...,[],25.411765,intro,../data/midi/transposed/hooktheory/pianoroll/w...,3/4,3,85,['Piano'],85.0,yu-gi-oh,https://www.hooktheory.com/theorytab/view/wayn...,wayne-sharpe,C major,C major
2,['chorus'],6.0,D,../data/midi/hooktheory/pianoroll/w/what-a-day...,['Jazz'],10.000000,chorus,../data/midi/transposed/hooktheory/pianoroll/w...,4/4,4,96,"['Piano', 'Piano']",96.0,kiefer,https://www.hooktheory.com/theorytab/view/what...,what-a-day,A minor,A minor
3,"['verse', 'pre-chorus', 'chorus']",6.0,D,../data/midi/hooktheory/pianoroll/w/whiteflame...,"['J-Pop', 'Pop']",25.263158,chorus,../data/midi/transposed/hooktheory/pianoroll/w...,4/4,4,152,"['Piano', 'Piano']",152.0,Senbonzakura,https://www.hooktheory.com/theorytab/view/whit...,whiteflame,D minor,D minor
4,"['verse', 'pre-chorus', 'chorus']",6.0,D,../data/midi/hooktheory/pianoroll/w/whiteflame...,"['J-Pop', 'Pop']",12.631579,verse,../data/midi/transposed/hooktheory/pianoroll/w...,4/4,4,152,"['Piano', 'Piano']",152.0,Senbonzakura,https://www.hooktheory.com/theorytab/view/whit...,whiteflame,D minor,D minor


In [8]:
df_filtered = df.loc[df['time_signature'] == '4/4']

In [22]:
def transcribe_file(midi_file):
    if not midi_file: return None
    out_file = Path(midi_file.replace('/transposed/', '/transcribedc_v1/')).with_suffix('.txt')
    out_file.parent.mkdir(parents=True, exist_ok=True)
    if out_file.exists(): return str(out_file)
    string_repr = midi2str(midi_file)
    with open(out_file, 'w') as tf:
        tf.write(string_repr)
    return str(out_file)

In [29]:
transcribed_files = {}
for idx,row in tqdm(df_filtered.iterrows(), total=df_filtered.shape[0]):
    transcribed_files[idx] = transcribe_file(row.midi_keyc)

100%|██████████| 11531/11531 [04:52<00:00,  5.16it/s] 


In [ ]:
# f1 = '../data/midi/transcribedc_v1/hooktheory/pianoroll/w/wayne-sharpe/yu-gi-oh-theme-song/chorus_key.txt'
# with open(f1, 'r') as f: 
#     print(f.read())

In [51]:
tdf = pd.DataFrame(data={'transcribed_keyc_v1': list(transcribed_files.values())}, index=transcribed_files.keys())

In [52]:
merged_df = df.join(tdf, how='outer')

In [53]:
tdf.shape, df.shape, merged_df.shape

((11531, 1), (12703, 18), (12703, 19))

In [56]:
merged_df.to_csv('hooktheory_metadata_transcribed.csv'); merged_df.head()

,parts,ht_mode,ht_key,original_path,genres,seconds,section,midi_keyc,time_signature,ht_time_signature,ht_bpm,instruments,bpm,midi_title,song_url,artist,inferred_key,inferred_keyc,transcribed_keyc_v1
0,"['intro', 'chorus']",1.0,C,../data/midi/hooktheory/pianoroll/w/wayne-shar...,[],15.000000,chorus,../data/midi/transposed/hooktheory/pianoroll/w...,4/4,4,128,"['Piano', 'Piano']",128.0,yu-gi-oh3,https://www.hooktheory.com/theorytab/view/wayn...,wayne-sharpe,C major,C major,../data/midi/transcribedc_v1/hooktheory/pianor...
1,"['intro', 'chorus']",1.0,C,../data/midi/hooktheory/pianoroll/w/wayne-shar...,[],25.411765,intro,../data/midi/transposed/hooktheory/pianoroll/w...,3/4,3,85,['Piano'],85.0,yu-gi-oh,https://www.hooktheory.com/theorytab/view/wayn...,wayne-sharpe,C major,C major,NaN
2,['chorus'],6.0,D,../data/midi/hooktheory/pianoroll/w/what-a-day...,['Jazz'],10.000000,chorus,../data/midi/transposed/hooktheory/pianoroll/w...,4/4,4,96,"['Piano', 'Piano']",96.0,kiefer,https://www.hooktheory.com/theorytab/view/what...,what-a-day,A minor,A minor,../data/midi/transcribedc_v1/hooktheory/pianor...
3,"['verse', 'pre-chorus', 'chorus']",6.0,D,../data/midi/hooktheory/pianoroll/w/whiteflame...,"['J-Pop', 'Pop']",25.263158,chorus,../data/midi/transposed/hooktheory/pianoroll/w...,4/4,4,152,"['Piano', 'Piano']",152.0,Senbonzakura,https://www.hooktheory.com/theorytab/view/whit...,whiteflame,D minor,D minor,../data/midi/transcribedc_v1/hooktheory/pianor...
4,"['verse', 'pre-chorus', 'chorus']",6.0,D,../data/midi/hooktheory/pianoroll/w/whiteflame...,"['J-Pop', 'Pop']",12.631579,verse,../data/midi/transposed/hooktheory/pianoroll/w...,4/4,4,152,"['Piano', 'Piano']",152.0,Senbonzakura,https://www.hooktheory.com/theorytab/view/whit...,whiteflame,D minor,D minor,../data/midi/transcribedc_v1/hooktheory/pianor...
